In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

from sklearn.model_selection import GridSearchCV

from datetime import datetime

pd.set_option('display.max_rows', 5000)

zhi_df2=pd.read_csv('../data/Zip_Zri_MultiFamilyResidenceRental.csv')


zri = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

#zri=zri[zri['year-month']>'2014-02']

zri=zri.fillna(0)
zri['gapsize']=zri['ZRI'].rolling(6).sum()
droplist=list(zri[(zri['gapsize']==0) & (zri['year-month']>'2014-06')]['RegionName'].drop_duplicates())
zri[(zri['gapsize']==0)].shape[0]/zri.shape[0]
len(droplist)/len(zri['RegionName'].drop_duplicates())

0.30037614185921546

In [2]:
zhi_df2.shape

(1861, 120)

In [3]:
date_mapping={i:j for i,j in enumerate(zhi_df2.columns[6:])}

In [4]:
zri2 = pd.melt(zhi_df2, id_vars = zhi_df2.columns[0:6], value_vars = zhi_df2.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionName', 'year-month'])

zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())]=zri2[(zri2['year-month']=='2019-12') & (zri2['ZRI'].isna())].fillna(method='ffill')

zri2=zri2.groupby('RegionName').apply(lambda group: group.interpolate())
#zri2[zri2['ZRI'].isna()]

#zri2['ZRI']=(zri2['ZRI']/zri2['ZRI'].shift(1))-1

zri2=zri2[(zri2['year-month']>='2014-01') & (zri2['year-month']<'2020-01')]

zri3=zri2[~(zri2['RegionName'].isin(droplist))]

zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')



<ipython-input-4-ec9c392b6a1a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zri3['ZRI']=zri3['ZRI'].fillna(method='bfill')


In [5]:
zri3['year']=zri3['year-month'].apply(lambda x: x[0:4])

<ipython-input-5-810f8ffd2dc5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zri3['year']=zri3['year-month'].apply(lambda x: x[0:4])


In [6]:
zri4=zri3.groupby(['RegionName','City','State','CountyName','year']).agg('mean')
zri4=zri4.reset_index()

In [7]:
#zip codes to drop
len(droplist)

559

In [8]:
#remaining filterd ZRI data:
len(zri4['RegionName'].drop_duplicates())

1302

In [9]:
zri4.isna().sum()

RegionName    0
City          0
State         0
CountyName    0
year          0
RegionID      0
ZRI           0
dtype: int64

<H1>Standardization

<H1>Lag development

In [10]:
zri4=zri4.drop('RegionID', axis=1)

In [11]:
def laggenerator(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i]
    
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    
    df.loc[df['year'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [12]:
def laggenerator_diff(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+1}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+1)
    
    df.loc[df['year'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+1}']=0
    return df

In [13]:
def laggenerator_diff6(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+6}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+6)
    
    df.loc[df['year'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+6}']=0
    return df

In [14]:
def laggenerator_MA(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i+4]
    
    df.loc[:,f'{colname}_MA{i}']=df.loc[:,f'{colname}'].shift(i)-(df.loc[:,f'{colname}'].shift(i)+df.loc[:,f'{colname}'].shift(i+1)+df.loc[:,f'{colname}'].shift(i+2)+df.loc[:,f'{colname}'].shift(i+3)+df.loc[:,f'{colname}'].shift(i+4))/5
    
    df.loc[df['year'].isin(timelist),f'{colname}_MA{i}']=0
    return df

In [15]:
def laggenerator_mean(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_mean{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_mean{i}']+=df.loc[:,f'{colname}'].shift(x)
    df.loc[:,f'{colname}_mean{i}']=df.loc[:,f'{colname}_mean{i}']/i
    
    df.loc[df['year'].isin(timelist),f'{colname}_mean{i}']=0
    return df

In [16]:
def laggenerator_ewm(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_ewm{i}']=0
    weightsum=0
    for y in range(1,i):
        weight=i-y
        weightsum+=weight
    for x in range(1,i):
        weight=i-x
        df.loc[:,f'{colname}_ewm{i}']+=df.loc[:,f'{colname}'].shift(x)*weight/weightsum
    df.loc[:,f'{colname}_ewm{i}']=df.loc[:,f'{colname}_ewm{i}']
    
    df.loc[df['year'].isin(timelist),f'{colname}_ewm{i}']=0
    return df

In [17]:
def laggenerator_min(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_min{i}']=999999
    for x in range(1,i):
        df.loc[:,f'{colname}_min{i}']=np.minimum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_min{i}'])
    
    df.loc[df['year'].isin(timelist),f'{colname}_min{i}']=0
    return df

In [18]:
def laggenerator_max(i,colname,df):
    timelist=list(df['year'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_max{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_max{i}']=np.maximum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_max{i}'])
    
    df.loc[df['year'].isin(timelist),f'{colname}_max{i}']=0
    return df

In [140]:
def lag_generator_all(df):
    for i in range(1,4):
        df=laggenerator(i, 'ZRI',df)

#     for i in range(1,3):
#         df=laggenerator_diff(i, 'ZRI',df)

#     for i in range(1,3):
#         df=laggenerator_diff6(i, 'ZRI',df)

#     for i in range(1,3):
#         df=laggenerator_MA(i, 'ZRI',df)

#     for i in range(1,3):
#         df=laggenerator_mean(i, 'ZRI',df)

    for i in range(1,4):
        df=laggenerator_ewm(i, 'ZRI',df)

#     for i in range(1,3):
#         df=laggenerator_min(i, 'ZRI',df)

#     for i in range(1,3):
#         df=laggenerator_max(i, 'ZRI',df)
    
    return df

In [141]:
zri4=lag_generator_all(zri4)


In [143]:
train, test=zri4[(zri4['year']<='2018') & (zri4['year']>'2015')], zri4[(zri4['year']>'2018') & (zri4['year']<='2019')]

train=train.drop('year',axis=1)

Y_train, Y_test=train['ZRI'], test[['ZRI']]
X_train, X_test=train.drop('ZRI',axis=1), test.drop('ZRI',axis=1)



In [144]:
labelencoder = LabelEncoder()
labelencoder2 = LabelEncoder()
labelencoder3 = LabelEncoder()
labelencoder4 = LabelEncoder()

#X_train['StateEnc'] = labelencoder.fit_transform(X_train['State'])
X_train['CountyNameEnc'] = labelencoder2.fit_transform(X_train['CountyName'])
#X_train['CityEnc'] = labelencoder3.fit_transform(X_train['City'])

X_train=X_train.drop(['CountyName','State','City','RegionName'],axis=1)


In [145]:
zips=zri4['RegionName'].unique()[0:5]
px.line(zri4[zri4['RegionName'].isin(zips)], y=['ZRI'],x='year', color='RegionName')

In [146]:
# cat=CatBoostRegressor(logging_level='Silent',loss_function='RMSE')

# params=({
#     'random_state':[0], 
#     'n_estimators':[500], 
#     'learning_rate':[0.09,0.1,0.12], 
#     'subsample':[0.8], 
#     'max_depth':[8]
# })

# grid=GridSearchCV(cat, param_grid=params, cv=5)
# grid.fit(X_train, Y_train)
#
#grid.best_params_
#
# Result:
#     {'learning_rate': 0.09,
#  'max_depth': 8,
#  'n_estimators': 500,
#  'random_state': 0,
#  'subsample': 0.8}

In [147]:
cat=CatBoostRegressor(logging_level='Silent',loss_function='RMSE', random_state=0, n_estimators=500, learning_rate=0.09, max_depth=8, subsample=0.8)
cat.fit(X_train, Y_train)

In [148]:
cat.score(X_train, Y_train)

0.9982774948828248

In [149]:

predictor_table=zri4.copy()
X_test=predictor_table[predictor_table['year']>'2018']


#run prediction for one month
X_test2=X_test.copy()

#X_test['StateEnc'] = labelencoder.transform(X_test['State'])
X_test2['CountyNameEnc'] = labelencoder2.transform(X_test2['CountyName'])
#X_test['CityEnc'] = labelencoder3.transform(X_test['City'])

X_test2=X_test2.drop(['CountyName','State','City','RegionName','year','ZRI'],axis=1)

X_test['ZRI_pred']=cat.predict(X_test2)





<ipython-input-149-ae28e6cba8af>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [150]:
RSME=(sum((X_test['ZRI']-X_test['ZRI_pred'])**2)/len(X_test))**0.5
RSME

69.88152463301633

In [151]:

RSME_null=(sum((X_test['ZRI']-X_test['ZRI_lag1'])**2)/len(X_test))**0.5
RSME_null


71.45587309315309